<a href="https://colab.research.google.com/github/ywang1110/Modeling_Triboelectric_Performance/blob/main/Humidity___PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import matplotlib.cm as cm
from IPython.display import display
from sklearn.feature_selection import f_regression

# Prepare dataset

## Load data

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
df = pd.read_csv('drive/My Drive/0 Sarah PP/0 Data/Humidity/Sarah_PP.xlsx')
df.head()

,Unnamed: 0,Label,ChargeDensity(µC/m2) j,Thickness(um),solidity,diameter(um),Interface(mm),Tem,Humidity (%),Force (N)
0,0,S-1,13.233000,524.766667,0.088333,6.184,1.123,25.133333,47.633333,36.133333
1,1,S-11,19.201667,360.533333,0.127333,3.296,0.765,24.766667,50.033333,36.433333
2,2,S-13,10.868000,537.533333,0.087667,6.238,1.429,25.733333,45.600000,36.766667
3,3,S-14,19.316333,404.166667,0.118667,3.580,0.752,24.600000,49.466667,35.800000
4,4,S-15,26.553667,316.933333,0.148667,2.554,0.857,25.600000,45.033333,36.100000


In [35]:
df.shape

(17, 10)

## Unit conversion

In [36]:
data = pd.DataFrame()
data['ChargeDensity(C/m2)']=df['ChargeDensity(µC/m2) j']*pow(10,-6)
data['thickness(m)']=df['Thickness(um)']*pow(10,-6)
data['solidity'] = df['solidity']
data['diameter(m)'] = df['diameter(um)']*pow(10,-6)
data['Interface(m)']=df['Interface(mm)']*pow(10, -3)
data[['Tem (°C)', 'Humidity (%)', 'Force (N)']]=df[['Tem', 'Humidity (%)', 'Force (N)']]

data.head()

,ChargeDensity(C/m2),thickness(m),solidity,diameter(m),Interface(m),Tem (°C),Humidity (%),Force (N)
0,0.000013,0.000525,0.088333,0.000006,0.001123,25.133333,47.633333,36.133333
1,0.000019,0.000361,0.127333,0.000003,0.000765,24.766667,50.033333,36.433333
2,0.000011,0.000538,0.087667,0.000006,0.001429,25.733333,45.600000,36.766667
3,0.000019,0.000404,0.118667,0.000004,0.000752,24.600000,49.466667,35.800000
4,0.000027,0.000317,0.148667,0.000003,0.000857,25.600000,45.033333,36.100000


In [37]:
data.shape

(17, 8)

# Train test split

In [38]:
random_state=76
kf = KFold(n_splits=5, shuffle = True, random_state=random_state)
train_indexs = []
test_indexs = []
for train_index, test_index in kf.split(data):
    train_indexs.append(train_index)
    test_indexs.append(test_index)
index=pd.DataFrame(zip(train_indexs, test_indexs), columns=['train_index','test_index'])
index

,train_index,test_index
0,"[0, 1, 2, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16]","[3, 5, 6, 11]"
1,"[0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 15, 16]","[4, 9, 13, 14]"
2,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]","[2, 8, 16]"
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16]","[0, 10, 12]"
4,"[0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16]","[1, 7, 15]"


In [39]:
n_fold=4

In [40]:
train_index = index.iloc[n_fold,0]
test_index = index.iloc[n_fold,1]
train, test = data.iloc[train_index], data.iloc[test_index]
train.shape

(14, 8)

In [41]:
train.shape

(14, 8)

In [42]:
test.shape

(3, 8)

In [43]:
"""
train.to_csv('train.csv')
test.to_csv('test.csv')
"""


"\ntrain.to_csv('train.csv')\ntest.to_csv('test.csv')\n"

In [44]:
"""
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
"""

"\ntrain = pd.read_csv('train.csv')\ntest=pd.read_csv('test.csv')\n"

In [45]:
train.head()

,ChargeDensity(C/m2),thickness(m),solidity,diameter(m),Interface(m),Tem (°C),Humidity (%),Force (N)
0,0.000013,0.000525,0.088333,0.000006,0.001123,25.133333,47.633333,36.133333
2,0.000011,0.000538,0.087667,0.000006,0.001429,25.733333,45.600000,36.766667
3,0.000019,0.000404,0.118667,0.000004,0.000752,24.600000,49.466667,35.800000
4,0.000027,0.000317,0.148667,0.000003,0.000857,25.600000,45.033333,36.100000
5,0.000015,0.000438,0.098333,0.000004,0.000886,26.166667,40.066667,35.866667


In [46]:
# Train the model
x=train['thickness(m)'].to_numpy()
y=train['solidity'].to_numpy()
z=train['diameter(m)'].to_numpy()
u=train['Interface(m)'].to_numpy()
h=train['Humidity (%)'].to_numpy()
j=train['ChargeDensity(C/m2)'].to_numpy()
def func(X,a, w, c, m, n, p):
  x,y,z,u,h = X
  return (a-w*h)*(2*x*y/z-c*y**m*x**n/z**p)*0.01/(0.01+u)

# lm
popt_lm,pcov = curve_fit(func,(x,y,z,u,h),j, method='lm', maxfev = 50000)
y_model_lm = func((x,y,z,u, h),popt_lm[0],popt_lm[1],popt_lm[2],popt_lm[3],popt_lm[4],popt_lm[5]) 
MSE_lm = mean_squared_error(j, y_model_lm)
MAE_lm = mean_absolute_error(j, y_model_lm)
lm_result="lm Method | MSE = {:.2E}| MAE = {:.2E}| a = {:.2E}, w = {:.2E}, c = {:.2E},m = {:.2E}, n = {:.2E}, p = {:.2E}".format(MSE_lm,MAE_lm,popt_lm[0],popt_lm[1],popt_lm[2],popt_lm[3],popt_lm[4],popt_lm[5])
lm_result  

'lm Method | MSE = 1.15E-12| MAE = 9.07E-07| a = -2.78E+01, w = 3.39E-01, c = 2.00E+00,m = 1.00E+00, n = 1.00E+00, p = 1.00E+00'

In [47]:
# initial guesses for a,b,c,m,n,p:
p0 = 1E-6 , 0 ,1. , 1. , 1., 1.

In [48]:
  # trf & dogbox
       #    a          w           c        m          n       p
  bound=((  0,         0,          0,       0,         0 ,     0 ), 
       (np.inf,    np.inf,    np.inf,     np.inf,    np.inf, np.inf))
  
  ## trf
  popt_trf,pcov = curve_fit(func,(x,y,z,u,h),j, p0, bounds = bound, method='trf', maxfev=50000)

  y_model_trf = func((x,y,z,u,h),popt_trf[0],popt_trf[1],popt_trf[2], popt_trf[3],popt_trf[4],popt_trf[5]) 
  MSE_trf = np.sum((j - y_model_trf)**2)/train.shape[0]
  MAE_trf = np.sum(abs(j - y_model_trf))/train.shape[0]

  ## dogbox
  popt_dog, pcov = curve_fit(func,(x,y,z,u,h),j, p0, bounds = bound, method='dogbox', maxfev=50000)
  y_model_dog = func((x,y,z,u,h),popt_dog[0],popt_dog[1],popt_dog[2],popt_dog[3],popt_dog[4],popt_dog[5]) 
  MSE_dog = np.sum((j - y_model_dog)**2)/train.shape[0]
  MAE_dog = np.sum(abs(j - y_model_dog))/train.shape[0]

  if MSE_trf < MSE_dog:
    a,w,c,m,n,p = popt_trf[0],popt_trf[1],popt_trf[2],popt_trf[3],popt_trf[4], popt_trf[5]
    print('trf Method is the best method')
  else:
    a,w,c,m,n,p = popt_dog[0], popt_dog[1], popt_dog[2], popt_dog[3], popt_dog[4], popt_dog[5]
    print("dogbox Method is the best method")
  trf_result="trf Method | MSE = {:.2E}| MAE = {:.2E}| a = {}, w = {}, c = {},m = {}, n = {}, p = {}".format(MSE_trf,MAE_trf,popt_trf[0],popt_trf[1],popt_trf[2],popt_trf[3],popt_trf[4], popt_trf[5])
  dog_result="dog Method | MSE = {:.2E}| MAE = {:.2E}| a = {}, w = {}, c = {},m = {}, n = {}, p = {}".format(MSE_dog,MAE_dog,popt_dog[0], popt_dog[1], popt_dog[2], popt_dog[3], popt_dog[4], popt_dog[5])
  print(trf_result)
  print(dog_result)

trf Method is the best method
trf Method | MSE = 1.94E-12| MAE = 1.23E-06| a = 5.321793810415626e-06, w = 1.824479736637613e-08, c = 1.0395832054009984,m = 1.0035655572234323, n = 1.140967847715325, p = 1.1255651923396084
dog Method | MSE = 1.38E-11| MAE = 3.28E-06| a = 4.726117539334488e-05, w = 5.818264939861959e-07, c = 1.9385986356554867,m = 1.0000136445536039, n = 1.0000324970447836, p = 1.0000154909778523


In [49]:
  # Validate (when trf is best)

  data_val = pd.concat([train, test])
  data_val.head()
  x=data_val['thickness(m)'].to_numpy()
  y=data_val['solidity'].to_numpy()
  z=data_val['diameter(m)'].to_numpy()
  u=data_val['Interface(m)'].to_numpy()
  h=data_val['Humidity (%)'].to_numpy()
  j=data_val['ChargeDensity(C/m2)'].to_numpy()


  data_val['Predicted(C/m2)'] = (a-w*h)*(2*x*y/z-c*y**m*x**n/z**p)*0.01/(0.01+u)

  data_val['ratio(%)'] = (data_val['Predicted(C/m2)'] -data_val['ChargeDensity(C/m2)'])/data_val['ChargeDensity(C/m2)']*100
  data_val['ChargeDensity(uC/m2)'] = data_val['ChargeDensity(C/m2)']*pow(10,6)
  data_val['Predicted(uC/m2)'] = (a-w*h)*(2*x*y/z-c*y**m*x**n/z**p)*0.01/(0.01+u)*pow(10,6)
  data_val['ratio_2(%)'] = (data_val['Predicted(uC/m2)']-data_val['ChargeDensity(uC/m2)'])/data_val['ChargeDensity(uC/m2)']*100
  def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  
  mape = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'], data_val['Predicted(C/m2)'])
  mape_train = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'][0:train.shape[0]], data_val['Predicted(C/m2)'][0:train.shape[0]])
  mape_test = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'][train.shape[0]:data_val.shape[0]], data_val['Predicted(C/m2)'][train.shape[0]:data_val.shape[0]])
  
  print('MAPE= {:.2f}'.format(mape))
  print('Train_MAPE = {:.2f}'.format(mape_train))
  print('test_MAPE = {:.2f}'.format(mape_test))

MAPE= 8.80
Train_MAPE = 7.98
test_MAPE = 12.63


# Validation at different random seeds

In [26]:
def get_index(random_state, data):
  kf = KFold(n_splits=5, shuffle = True, random_state=random_state)
  train_indexs = []
  test_indexs = []
  for train_index, test_index in kf.split(data):
      train_indexs.append(train_index)
      test_indexs.append(test_index)
  index=pd.DataFrame(zip(train_indexs, test_indexs), columns=['train_index','test_index'])
  return index

In [27]:
def get_scores(n_fold, index, data):
  train_index = index.iloc[n_fold,0]
  test_index = index.iloc[n_fold,1]
  train, test = data.iloc[train_index], data.iloc[test_index]
  train_shape=train.shape
  test_shape=test.shape
  
  # Validate (when trf is best)
  a,w,c,m,n,p = popt_trf[0],popt_trf[1],popt_trf[2], popt_trf[3],popt_trf[4],popt_trf[5]
  
  data_val = pd.concat([train, test])
  x=data_val['thickness(m)'].to_numpy()
  y=data_val['solidity'].to_numpy()
  z=data_val['diameter(m)'].to_numpy()
  u=data_val['Interface(m)'].to_numpy()
  j=data_val['ChargeDensity(C/m2)'].to_numpy()


  data_val['Predicted(C/m2)'] = (a-w*h)*(2*x*y/z-c*y**m*x**n/z**p)*0.01/(0.01+u)

  
  def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  
  mape_trf = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'], data_val['Predicted(C/m2)'])
  mape_trf_train = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'][0:train.shape[0]], data_val['Predicted(C/m2)'][0:train.shape[0]])
  mape_trf_test = mean_absolute_percentage_error(data_val['ChargeDensity(C/m2)'][train.shape[0]:data_val.shape[0]], data_val['Predicted(C/m2)'][train.shape[0]:data_val.shape[0]])

  return n_fold, train_shape, test_shape, mape_trf, mape_trf_train, mape_trf_test

In [28]:
max_random_sate = 100

In [29]:
random_states = []
n_folds = []
train_shapes = []
test_shapes = []

MAPEs = []
train_MAPEs = []
test_MAPEs = []


for random_state in range(max_random_sate):
  index = get_index(random_state, data)
  for n_fold in range(5):
    random_states.append(random_state)
    n_fold, train_shape, test_shape, MAPE, train_MAPE, test_MAPE = get_scores(n_fold, index, data)
    n_folds.append(n_fold)
    train_shapes.append(train_shape)
    test_shapes.append(test_shape)
    
    MAPEs.append(MAPE)
    train_MAPEs.append(train_MAPE)
    test_MAPEs.append(test_MAPE)


In [30]:
summary = pd.DataFrame()
summary['random_state']=random_states
summary['n_fold']=n_folds
summary['train_shape'] = train_shapes
summary['test_shape'] = test_shapes

summary['MAPE'] = MAPEs
summary['train_MAPE'] = train_MAPEs
summary['test_MAPE'] = test_MAPEs

summary

,random_state,n_fold,train_shape,test_shape,MAPE,train_MAPE,test_MAPE
0,0,0,"(13, 8)","(4, 8)",8.854322,9.487861,6.795321
1,0,1,"(13, 8)","(4, 8)",7.737251,8.101274,6.554177
2,0,2,"(14, 8)","(3, 8)",7.135930,6.187431,11.562258
3,0,3,"(14, 8)","(3, 8)",7.818146,7.859989,7.622875
4,0,4,"(14, 8)","(3, 8)",7.025799,7.143708,6.475561
...,...,...,...,...,...,...,...
495,99,0,"(13, 8)","(4, 8)",8.407271,7.140729,12.523532
496,99,1,"(13, 8)","(4, 8)",7.234328,6.866539,8.429642
497,99,2,"(14, 8)","(3, 8)",8.992134,10.058446,4.016008
498,99,3,"(14, 8)","(3, 8)",7.015079,7.424254,5.105595


In [31]:
summary.describe()

,random_state,n_fold,MAPE,train_MAPE,test_MAPE
count,500.000000,500.00000,500.000000,500.000000,500.000000
mean,49.500000,2.00000,8.012681,8.096592,7.705742
std,28.894979,1.41563,0.686765,0.996347,3.278972
min,0.000000,0.00000,6.714170,5.368286,0.673929
25%,24.750000,1.00000,7.355626,7.429535,5.392487
50%,49.500000,2.00000,8.153099,8.092473,7.614480
75%,74.250000,3.00000,8.517846,8.829112,9.735625
max,99.000000,4.00000,9.807130,10.575909,16.697582


In [51]:
summary.to_csv('100RandomState_PP.csv')